# Recherche Wikipedia

Le but ici est de permettre à l'utilisateur d'effectuer une recherche wikipédia à partir d'un mot clé

In [15]:
import requests

def rechercher_wikipedia(terme):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{terme}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        titre = data.get("title", "Sans titre")
        extrait = data.get("extract", "Pas de résumé trouvé.")
        url_article = data.get("content_urls", {}).get("desktop", {}).get("page", "#")

        return {
            "titre": titre,
            "extrait": extrait,
            "url": url_article
        }

    except requests.exceptions.HTTPError:
        return {
            "erreur": f"Aucun article trouvé pour « {terme} »."
        }
    except Exception as e:
        return {
            "erreur": f"Erreur inattendue : {e}"
        }

In [16]:
rechercher_wikipedia("Kobe Bryant")

{'titre': 'Kobe Bryant',
 'extrait': 'Kobe Bean Bryant was an American professional basketball player. A shooting guard, he spent his entire 20-year career with the Los Angeles Lakers in the National Basketball Association (NBA). Widely regarded as one of the greatest and most influential basketball players of all time, Bryant won five NBA championships and was an 18-time All-Star, four-time All-Star MVP, 15-time member of the All-NBA Team, 12-time member of the All-Defensive Team, the 2008 NBA Most Valuable Player (MVP), two-time NBA Finals MVP, and two-time scoring champion. He ranks fourth in league all-time regular season and postseason scoring. Bryant was posthumously named to the NBA 75th Anniversary Team in 2021 and was inducted to the Naismith Memorial Basketball Hall of Fame twice, first in 2020 as an NBA player, and again in 2025 as a member of the 2008 U.S. Olympic team.',
 'url': 'https://en.wikipedia.org/wiki/Kobe_Bryant'}

In [3]:
def recuperer_article_wikipedia(titre):
    """
    Récupère le contenu complet brut (texte seul) d'un article Wikipédia.
    """
    url = "https://fr.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": titre,
        "exintro": True,
        "explaintext": True,
        "redirects": 1  # Pour suivre les redirections
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        pages = data["query"]["pages"]
        page = next(iter(pages.values()))

        if "extract" in page and page["extract"].strip():
            return page["extract"]
        else:
            return None

    except Exception as e:
        print(f"Erreur lors de la récupération de l'article : {e}")
        return None

In [4]:
article = (recuperer_article_wikipedia("napoleon"))
print(article)

Napoléon Bonaparte, né le 15 août 1769 à Ajaccio en Corse et mort le 5 mai 1821 à Longwood sur l’île de Sainte-Hélène, est un militaire et homme d'État français. Il est le premier empereur des Français du 18 mai 1804 au 6 avril 1814 et du 20 mars au 22 juin 1815, sous le nom de Napoléon Ier.
Second enfant de Charles Bonaparte et Letizia Ramolino, Napoléon Bonaparte devient en 1793 général dans les armées de la Première République française, née de la Révolution, où il est notamment commandant en chef de l'armée d'Italie puis de l'armée d'Orient. Arrivé au pouvoir en 1799 par le coup d'État du 18 Brumaire, il est Premier consul — consul à vie à partir du 2 août 1802 — jusqu'au 18 mai 1804, date à laquelle l'Empire est proclamé par un sénatus-consulte suivi d'un plébiscite. Il est sacré empereur, en la cathédrale Notre-Dame de Paris, le 2 décembre 1804, par le pape Pie VII, en même temps que son épouse Joséphine de Beauharnais.
En tant que général en chef et chef d'État, Napoléon tente d

In [5]:
import joblib
# Chargement du Sumarizer
summarizer = joblib.load("ML_Sumarizer/summarize_pipeline_XGBoost.pkl")

In [6]:
from nltk.tokenize import sent_tokenize
from utils import preprocessing, encode_text
import scipy
import numpy as np

sentences = sent_tokenize(article)
cleaned_sentences = [preprocessing(s) for s in sentences]

vectorizer = summarizer.named_steps['vectorizer']
clf = summarizer.named_steps['classifier']
X_vec = vectorizer.transform(cleaned_sentences)

if hasattr(clf, "predict_proba"):
    probas = clf.predict_proba(X_vec)[:, 1]
elif hasattr(clf, "decision_function"):
    decision_scores = clf.decision_function(X_vec)
    probas = scipy.special.softmax(np.vstack([-decision_scores, decision_scores]), axis=0)[1]
else:
    raise ValueError("Le modèle ne supporte ni predict_proba ni decision_function")

top_k = 5
top_k_idx = np.argsort(probas)[-top_k:][::-1]
summary_sentences = [sentences[i] for i in sorted(top_k_idx)]

[nltk_data] Downloading package punkt to /Users/hugo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/hugo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/hugo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hugo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/hugo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [7]:
summary_sentences

["Second enfant de Charles Bonaparte et Letizia Ramolino, Napoléon Bonaparte devient en 1793 général dans les armées de la Première République française, née de la Révolution, où il est notamment commandant en chef de l'armée d'Italie puis de l'armée d'Orient.",
 "En tant que général en chef et chef d'État, Napoléon tente de briser les coalitions montées et financées par le royaume de Grande-Bretagne et qui rassemblent, à partir de 1792, les monarchies européennes contre la France et son régime né de la Révolution.",
 "Ses victoires lui permettent d'annexer à la France de vastes territoires et de gouverner la majeure partie de l'Europe continentale en plaçant les membres de sa famille sur les trônes de plusieurs royaumes : Joseph à Naples puis en Espagne, Louis en Hollande, Jérôme en Westphalie et son beau-frère Joachim Murat à Naples.",
 "Amené à abdiquer en 1814 après la prise de Paris, capitale de l'Empire français, et à se retirer à l'île d'Elbe, il tente de reprendre le pouvoir en

In [8]:
from utils import summarize_tf_idf, summarize_tf_idf_with_scores

sum_tfidf = summarize_tf_idf(article)
sum_score = summarize_tf_idf_with_scores(article)

In [9]:
sum_tfidf

"Ses victoires lui permettent d'annexer à la France de vastes territoires et de gouverner la majeure partie de l'Europe continentale en plaçant les membres de sa famille sur les trônes de plusieurs royaumes : Joseph à Naples puis en Espagne, Louis en Hollande, Jérôme en Westphalie et son beau-frère Joachim Murat à Naples. Capable de reconquérir la France et d'y rétablir le régime impérial sans coup férir, il amène pourtant, à la suite de diverses trahisons et dissensions de ses maréchaux, le pays dans une impasse avec la lourde défaite de Waterloo, qui met fin à l'Empire napoléonien et assure la restauration de la dynastie des Bourbons. Objet dès son vivant d'une légende dorée comme d'une légende noire, il doit sa très grande notoriété à son habileté militaire, récompensée par de nombreuses victoires, et à sa trajectoire politique étonnante, mais aussi à son régime despotique et très centralisé ainsi qu'à son ambition, qui se traduit par des guerres meurtrières (au Portugal, en Espagne

In [10]:
sum_score[0]

"Ses victoires lui permettent d'annexer à la France de vastes territoires et de gouverner la majeure partie de l'Europe continentale en plaçant les membres de sa famille sur les trônes de plusieurs royaumes : Joseph à Naples puis en Espagne, Louis en Hollande, Jérôme en Westphalie et son beau-frère Joachim Murat à Naples. Capable de reconquérir la France et d'y rétablir le régime impérial sans coup férir, il amène pourtant, à la suite de diverses trahisons et dissensions de ses maréchaux, le pays dans une impasse avec la lourde défaite de Waterloo, qui met fin à l'Empire napoléonien et assure la restauration de la dynastie des Bourbons. Objet dès son vivant d'une légende dorée comme d'une légende noire, il doit sa très grande notoriété à son habileté militaire, récompensée par de nombreuses victoires, et à sa trajectoire politique étonnante, mais aussi à son régime despotique et très centralisé ainsi qu'à son ambition, qui se traduit par des guerres meurtrières (au Portugal, en Espagne

In [12]:
from utils import summarize_article
print(summarize_article(article, summarizer, preprocessing, top_k=3))

Second enfant de Charles Bonaparte et Letizia Ramolino, Napoléon Bonaparte devient en 1793 général dans les armées de la Première République française, née de la Révolution, où il est notamment commandant en chef de l'armée d'Italie puis de l'armée d'Orient.

En tant que général en chef et chef d'État, Napoléon tente de briser les coalitions montées et financées par le royaume de Grande-Bretagne et qui rassemblent, à partir de 1792, les monarchies européennes contre la France et son régime né de la Révolution.

Ses victoires lui permettent d'annexer à la France de vastes territoires et de gouverner la majeure partie de l'Europe continentale en plaçant les membres de sa famille sur les trônes de plusieurs royaumes : Joseph à Naples puis en Espagne, Louis en Hollande, Jérôme en Westphalie et son beau-frère Joachim Murat à Naples.
